In [1]:
# Speed comparison SZ
import pandas as pd
import numpy as np
import glob
import os

path = np.array(glob.glob('/mnt/data/20201221/logs_20201221_zs_96_03_day_96data.tar.gz/mdLog_SZ_***'))
assert(len(path) == 1)
print(path[0])
path1 = np.array(glob.glob('/mnt/data/20201221/zt_7401_20201221_10.235.16.80.tar.gz/Logs_full/mdLog_SZ_***'))
assert(len(path1) == 1)
print(path1[0])
logSZ = pd.read_csv(path[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
logSZ1 = pd.read_csv(path1[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
print("original data source type: ")
print(logSZ["source"].unique())
print("new data source type: ")
print(logSZ1["source"].unique())
print(logSZ["time"].unique())
print(logSZ1["time"].unique())
assert(logSZ['source'].nunique() == 1)
assert(logSZ1['source'].nunique() == 1)

logSZ1["time"] = logSZ1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))

display(logSZ['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())
display(logSZ1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())

if (logSZ['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).nunique() == 1) | \
(logSZ1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).nunique() == 1):
    print('cum_amount precision is insufficient')
    columns = ["StockID", "cum_volume", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades"]
else:
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
       "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
       "ask4q", "ask5q", "openPrice", "numTrades"]

data1 = logSZ[(logSZ["cum_volume"] > 0)]
data2 = logSZ1[(logSZ1["cum_volume"] > 0)]

data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())
print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]
data1_1['cum_amount'] = data1_1['cum_amount'].round(2)
data2_1['cum_amount'] = data2_1['cum_amount'].round(2)
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])
    print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
    print(test[np.isnan(test["sequenceNo_y"])]['time_x'].astype(int).unique())
    print(test[np.isnan(test["sequenceNo_y"])]['StockID'].astype(int).unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["sequenceNo_x"])])
    print(test[np.isnan(test["sequenceNo_x"])]['time_y'].astype(int).unique())
    print(test[np.isnan(test["sequenceNo_x"])]['StockID'].astype(int).unique())
    display(n2-n1)

/mnt/data/20201221/logs_20201221_zs_96_03_day_96data.tar.gz/mdLog_SZ_20201221_0831.csv
/mnt/data/20201221/zt_7401_20201221_10.235.16.80.tar.gz/Logs_full/mdLog_SZ_20201221_0900.csv
original data source type: 
[24]
new data source type: 
[29]
['08:32:18.000' '08:33:18.000' '08:34:18.000' ... '15:24:03.000'
 '15:25:03.000' '15:26:03.000']
['09:01:18.000' '09:02:18.000' '09:03:18.000' ... '15:11:03.000'
 '15:11:15.000' '15:11:30.000']


array([1, 2, 6, 7])

array([1])

cum_amount precision is insufficient
2323
2130
7075794
7076242
7076242
-----------------------------------------------


'baseline is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount_x,close,...,ask5q,openPrice,numTrades,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange_y,time_y,cum_amount_y
7075794,NaN,NaN,NaN,NaN,300001,NaN,NaN,500,NaN,26.76,...,0,0.0,1,7768390,1608534285475710,115489078,29,SZ,150500000,13380.0
7075795,NaN,NaN,NaN,NaN,300002,NaN,NaN,200,NaN,5.49,...,0,0.0,1,7768391,1608534285475713,115489079,29,SZ,150500000,1098.0
7075796,NaN,NaN,NaN,NaN,300012,NaN,NaN,400,NaN,26.20,...,0,0.0,1,7768401,1608534285475948,115489089,29,SZ,150500000,10480.0
7075797,NaN,NaN,NaN,NaN,300014,NaN,NaN,1100,NaN,76.63,...,0,0.0,5,7768403,1608534285475950,115489091,29,SZ,150500000,84293.0
7075798,NaN,NaN,NaN,NaN,300015,NaN,NaN,100,NaN,69.44,...,0,0.0,1,7768404,1608534285476074,115489092,29,SZ,150500000,6944.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7076237,NaN,NaN,NaN,NaN,300346,NaN,NaN,300,NaN,41.23,...,0,0.0,3,7788947,1608534676378977,115519938,29,SZ,151130000,12369.0
7076238,NaN,NaN,NaN,NaN,300389,NaN,NaN,2700,NaN,12.05,...,0,0.0,3,7788950,1608534676660307,115519941,29,SZ,151130000,32535.0
7076239,NaN,NaN,NaN,NaN,300419,NaN,NaN,2500,NaN,8.04,...,0,0.0,4,7788953,1608534676660455,115519944,29,SZ,151130000,20100.0
7076240,NaN,NaN,NaN,NaN,300433,NaN,NaN,8400,NaN,29.94,...,0,0.0,12,7788955,1608534676668896,115519946,29,SZ,151130000,251496.0


ValueError: Cannot convert non-finite values (NA or inf) to integer

In [5]:
pd.set_option('max_columns', 200)
test[test['sequenceNo_x'].isnull()][columns + ['time_y']]

,StockID,cum_volume,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades,time_y
7075794,300001,500,26.76,26.76,0.0,0.0,0.0,0.0,0,0,0,0,0,26.76,0.0,0.0,0.0,0.0,800,0,0,0,0,0.0,1,150500000
7075795,300002,200,5.49,5.49,0.0,0.0,0.0,0.0,0,0,0,0,0,5.49,0.0,0.0,0.0,0.0,71500,0,0,0,0,0.0,1,150500000
7075796,300012,400,26.20,26.20,0.0,0.0,0.0,0.0,200,0,0,0,0,26.20,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,1,150500000
7075797,300014,1100,76.63,76.63,0.0,0.0,0.0,0.0,0,0,0,0,0,76.63,0.0,0.0,0.0,0.0,100,0,0,0,0,0.0,5,150500000
7075798,300015,100,69.44,69.44,0.0,0.0,0.0,0.0,0,0,0,0,0,69.44,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,1,150500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7076237,300346,300,41.23,41.23,0.0,0.0,0.0,0.0,8100,0,0,0,0,41.23,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,3,151130000
7076238,300389,2700,12.05,12.05,0.0,0.0,0.0,0.0,2100,0,0,0,0,12.05,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,3,151130000
7076239,300419,2500,8.04,8.04,0.0,0.0,0.0,0.0,0,0,0,0,0,8.04,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,4,151130000
7076240,300433,8400,29.94,29.94,0.0,0.0,0.0,0.0,0,0,0,0,0,29.94,0.0,0.0,0.0,0.0,8600,0,0,0,0,0.0,12,151130000


In [6]:
# Speed comparison SZ
import pandas as pd
import numpy as np
import glob
import os

path = np.array(glob.glob('/mnt/data/20201221/logs_20201221_zs_96_03_day_96data.tar.gz/mdLog_SZ_***'))
assert(len(path) == 1)
print(path[0])
path1 = np.array(glob.glob('/mnt/data/20201221/zt_7401_20201221_10.235.16.168.tar.gz/Logs_full/mdLog_SZ_***'))
assert(len(path1) == 1)
print(path1[0])
logSZ = pd.read_csv(path[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
logSZ1 = pd.read_csv(path1[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
print("original data source type: ")
print(logSZ["source"].unique())
print("new data source type: ")
print(logSZ1["source"].unique())
print(logSZ["time"].unique())
print(logSZ1["time"].unique())
assert(logSZ['source'].nunique() == 1)
assert(logSZ1['source'].nunique() == 1)

logSZ1["time"] = logSZ1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))

display(logSZ['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())
display(logSZ1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())

if (logSZ['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).nunique() == 1) | \
(logSZ1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).nunique() == 1):
    print('cum_amount precision is insufficient')
    columns = ["StockID", "cum_volume", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades"]
else:
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
       "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
       "ask4q", "ask5q", "openPrice", "numTrades"]

data1 = logSZ[(logSZ["cum_volume"] > 0)]
data2 = logSZ1[(logSZ1["cum_volume"] > 0)]

data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())
print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]
data1_1['cum_amount'] = data1_1['cum_amount'].round(2)
data2_1['cum_amount'] = data2_1['cum_amount'].round(2)
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])
    print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
    print(test[np.isnan(test["sequenceNo_y"])]['time_x'].astype(int).unique())
    print(test[np.isnan(test["sequenceNo_y"])]['StockID'].astype(int).unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["sequenceNo_x"])])
    print(test[np.isnan(test["sequenceNo_x"])]['time_y'].astype(int).unique())
    print(test[np.isnan(test["sequenceNo_x"])]['StockID'].astype(int).unique())
    display(n2-n1)

/mnt/data/20201221/logs_20201221_zs_96_03_day_96data.tar.gz/mdLog_SZ_20201221_0831.csv
/mnt/data/20201221/zt_7401_20201221_10.235.16.168.tar.gz/Logs_full/mdLog_SZ_20201221_0900.csv
original data source type: 
[24]
new data source type: 
[29]
['08:32:18.000' '08:33:18.000' '08:34:18.000' ... '15:24:03.000'
 '15:25:03.000' '15:26:03.000']
['09:01:18.000' '09:02:18.000' '09:03:18.000' ... '15:11:03.000'
 '15:11:15.000' '15:11:30.000']


array([1, 2, 6, 7])

array([1])

cum_amount precision is insufficient
2323
2130
7075794
7076242
7076242
-----------------------------------------------


'baseline is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount_x,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange_y,time_y,cum_amount_y
7075794,NaN,NaN,NaN,NaN,300001,NaN,NaN,500,NaN,26.76,26.76,0.0,0.0,0.0,0.0,0,0,0,0,0,26.76,0.0,0.0,0.0,0.0,800,0,0,0,0,0.0,1,7768390,1608534285475684,115488795,29,SZ,150500000,13380.0
7075795,NaN,NaN,NaN,NaN,300002,NaN,NaN,200,NaN,5.49,5.49,0.0,0.0,0.0,0.0,0,0,0,0,0,5.49,0.0,0.0,0.0,0.0,71500,0,0,0,0,0.0,1,7768391,1608534285475688,115488796,29,SZ,150500000,1098.0
7075796,NaN,NaN,NaN,NaN,300012,NaN,NaN,400,NaN,26.20,26.20,0.0,0.0,0.0,0.0,200,0,0,0,0,26.20,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,1,7768401,1608534285475910,115488806,29,SZ,150500000,10480.0
7075797,NaN,NaN,NaN,NaN,300014,NaN,NaN,1100,NaN,76.63,76.63,0.0,0.0,0.0,0.0,0,0,0,0,0,76.63,0.0,0.0,0.0,0.0,100,0,0,0,0,0.0,5,7768403,1608534285475922,115488808,29,SZ,150500000,84293.0
7075798,NaN,NaN,NaN,NaN,300015,NaN,NaN,100,NaN,69.44,69.44,0.0,0.0,0.0,0.0,0,0,0,0,0,69.44,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,1,7768404,1608534285475923,115488809,29,SZ,150500000,6944.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7076237,NaN,NaN,NaN,NaN,300346,NaN,NaN,300,NaN,41.23,41.23,0.0,0.0,0.0,0.0,8100,0,0,0,0,41.23,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,3,7788947,1608534676378962,115519655,29,SZ,151130000,12369.0
7076238,NaN,NaN,NaN,NaN,300389,NaN,NaN,2700,NaN,12.05,12.05,0.0,0.0,0.0,0.0,2100,0,0,0,0,12.05,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,3,7788950,1608534676660455,115519658,29,SZ,151130000,32535.0
7076239,NaN,NaN,NaN,NaN,300419,NaN,NaN,2500,NaN,8.04,8.04,0.0,0.0,0.0,0.0,0,0,0,0,0,8.04,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,4,7788953,1608534676668908,115519661,29,SZ,151130000,20100.0
7076240,NaN,NaN,NaN,NaN,300433,NaN,NaN,8400,NaN,29.94,29.94,0.0,0.0,0.0,0.0,0,0,0,0,0,29.94,0.0,0.0,0.0,0.0,8600,0,0,0,0,0.0,12,7788955,1608534676668943,115519663,29,SZ,151130000,251496.0


ValueError: Cannot convert non-finite values (NA or inf) to integer